In [6]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.optimizers import Adam

file_path = '/content/CancerFactorsData.xlsx'
df = pd.read_excel(file_path)

In [7]:
# Define your input features and target variable
features = ['Smoking', 'Obesity', 'Poverty', 'Uninsured', 'PM2.5']
target = 'LungCancerRates'

# Ensure data is sorted by time
df = df.sort_index()

# Normalize the features and the target separately
scaler_features = MinMaxScaler()
scaler_target = MinMaxScaler()

df[features] = scaler_features.fit_transform(df[features])
df[[target]] = scaler_target.fit_transform(df[[target]])

# Creating input sequences for LSTM
def create_sequences(df, features, target, sequence_length=10):
    X = []
    y = []
    for i in range(len(df) - sequence_length):
        X.append(df[features].iloc[i:i+sequence_length].values)
        y.append(df[target].iloc[i+sequence_length])
    return np.array(X), np.array(y)

sequence_length = 10
X, y = create_sequences(df, features, target, sequence_length)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Creating Sequential model
model = Sequential()

# Adding LSTM layers with Dropout for regularization
model.add(LSTM(64, input_shape=(sequence_length, len(features)), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(32))
model.add(Dropout(0.2))

# Adding Dense layer for regression output
model.add(Dense(1))

# Compiling the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['mae'])

# Training the model
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), verbose=1)

# Evaluating the model
loss, mae = model.evaluate(X_test, y_test)

# Making predictions
predictions = model.predict(X_test)

# Inverse transform the target values
y_test_original = scaler_target.inverse_transform(y_test.reshape(-1, 1))
predictions_original = scaler_target.inverse_transform(predictions)

# Calculating and printing metrics
mae = mean_absolute_error(y_test_original, predictions_original)
mse = mean_squared_error(y_test_original, predictions_original)
r2 = r2_score(y_test_original, predictions_original)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"R-squared (R²): {r2}")

# Printing sample predicted and actual values for comparison
print("Sample predictions and actual values:")
for i in range(5):  # Print first 5 samples for comparison
    print(f"Predicted: {predictions_original[i][0]}, Actual: {y_test_original[i][0]}")

Epoch 1/100
3/3 [==============================] - 5s 468ms/step - loss: 0.2418 - mae: 0.4468 - val_loss: 0.1844 - val_mae: 0.3338
Epoch 2/100
3/3 [==============================] - 0s 27ms/step - loss: 0.1088 - mae: 0.2725 - val_loss: 0.1045 - val_mae: 0.2471
Epoch 3/100
3/3 [==============================] - 0s 25ms/step - loss: 0.0526 - mae: 0.1891 - val_loss: 0.1001 - val_mae: 0.2680
Epoch 4/100
3/3 [==============================] - 0s 26ms/step - loss: 0.0650 - mae: 0.2117 - val_loss: 0.1082 - val_mae: 0.2826
Epoch 5/100
3/3 [==============================] - 0s 24ms/step - loss: 0.0661 - mae: 0.2040 - val_loss: 0.0969 - val_mae: 0.2649
Epoch 6/100
3/3 [==============================] - 0s 26ms/step - loss: 0.0530 - mae: 0.1856 - val_loss: 0.0910 - val_mae: 0.2521
Epoch 7/100
3/3 [==============================] - 0s 25ms/step - loss: 0.0530 - mae: 0.1834 - val_loss: 0.0921 - val_mae: 0.2435
Epoch 8/100
3/3 [==============================] - 0s 26ms/step - loss: 0.0482 - mae: 0.1